<a href="https://colab.research.google.com/github/minshyee/Section-4/blob/main/Get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data
## '방역패스', '백신패스' 에 대한 감성분석
### Naver News

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 958 kB 5.0 MB/s 
     |████████████████████████████████| 356 kB 42.0 MB/s 
     |████████████████████████████████| 138 kB 78.1 MB/s 
     |████████████████████████████████| 3.6 MB 59.4 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/comp

# crawler_naver news

In [23]:
# crawler_naver news
# Step 1. 네이버 > 검색어 입력 > 뉴스> 네이버뉴스 url 수집 > 저장 (naver_news_url.xlsx)
# Step 2. naver_news_url.xlsx 불러오기 > 제목, 날짜, 댓글, 좋아요 크롤링 > 저장 (naver_news_keyword.xlsx)

# pwd

### Step 0. 준비
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver import ActionChains as AC  # 웹 브라우저 동작
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time

### Step 1. 크롤링할 뉴스들 url 수집
# 검색어
keyword = "방역패스"

# 크롬 웹브라우저 실행
chrome_options = webdriver.ChromeOptions() # 객체 가져오기
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start=21".format(keyword))
time.sleep(2)

# page1에서 네이버뉴스 url 수집하기
things = driver.find_elements_by_link_text('네이버뉴스')  

url_list = []
for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

print(len(url_list))
# url_list

# url_list 저장
df = pd.DataFrame({"url":url_list})
# df.to_csv('naver_news_urls.xlsx')
df


크롤링할 키워드를 입력하세요: 방역패스


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


21


,url
0,https://news.naver.com/main/read.naver?mode=LS...
1,https://news.naver.com/main/read.naver?mode=LS...
2,https://news.naver.com/main/read.naver?mode=LS...
3,https://news.naver.com/main/read.naver?mode=LS...
4,https://news.naver.com/main/read.naver?mode=LS...
5,https://news.naver.com/main/read.naver?mode=LS...
6,https://news.naver.com/main/read.naver?mode=LS...
7,https://news.naver.com/main/read.naver?mode=LS...
8,https://news.naver.com/main/read.naver?mode=LS...
9,https://news.naver.com/main/read.naver?mode=LS...


In [24]:

### Step 2. 전체 기사 for문으로 데이터 수집
# 저장해둔 url 불러오기
print(len(df['url']))
# df['url']

number = len(df['url'])
print(number)

dict = {}

# 페이지당 기사 수집
for i in tqdm(range(0, number)):   # len(df['url'])
    try:
        # 뉴스 크롬창 띄우기
        driver = webdriver.Chrome('chromedriver', options=chrome_options)
        driver.get(df['url'][i])
        time.sleep(1)

        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tts_head').text
        date = driver.find_element_by_css_selector('.t11').text
        up = driver.find_element_by_css_selector('.u_cnt._count').text
        like = driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count").text
        warm = driver.find_element_by_css_selector(".end_btn .u_likeit_list.warm .u_likeit_list_count._count").text
        sad = driver.find_element_by_css_selector(".end_btn .u_likeit_list.sad .u_likeit_list_count._count").text
        angry = driver.find_element_by_css_selector(".end_btn .u_likeit_list.angry .u_likeit_list_count._count").text
        want = driver.find_element_by_css_selector(".end_btn .u_likeit_list.want .u_likeit_list_count._count").text

        # 기사 댓글 갯수 
        review_count = int(driver.find_element_by_css_selector(".u_cbox_count").text.replace(',', ''))

        # 더보기 클릭 횟수
        moreview_num = review_count//20

        # 댓글 버튼 클릭
        driver.find_element_by_css_selector(".lo_txt").click()
        time.sleep(1)

        # 더보기 버튼 여러번 클릭하기
        k=0
        while k <= moreview_num:  # 더보기 횟수만큼 반복 
            try:
                driver.find_element_by_css_selector(".u_cbox_page_more").click()  # 더보기 버튼 클릭
                time.sleep(1)
                k = k+1
            except:        
                break   # 에러나면 클릭 반복문을 빠져나가라

        # review 수집하기
        review_list = []
        overlays1 = ".u_cbox_text_wrap"
        reviews = driver.find_elements_by_css_selector(overlays1)
        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['추천수'] = up
        target_info['좋아요'] = like
        target_info['훈훈해요'] = warm
        target_info['슬퍼요'] = sad
        target_info['화나요'] = angry
        target_info['후속기사 원해요'] = want
        target_info['댓글 수'] = len(review_list)
        target_info['댓글'] = review_list

        dict[i] = target_info
        
        print(title, '( 댓글 수:', len(review_list),')')

        driver.close()
        time.sleep(1)
        
    except:
        driver.close()
        continue

print(len(dict))
# dict

# 판다스화
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

# 엑셀 저장
# result_df.to_excel("naver_news_({}).xlsx".format(keyword))

21
21


  0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis

  0%|          | 0/37 [00:00<?, ?it/s]

학원 방역패스 제동에…청소년 접종률 상승세 꺾였다 ( 댓글 수: 37 )


...

내일부터 백화점·대형마트 방역패스…17일부터 과태료 부과 ( 댓글 수: 268 )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis

  0%|          | 0/866 [00:00<?, ?it/s]

딩동댕~ 딩동~…백화점·마트 방역패스 확인 ‘북새통’ ( 댓글 수: 866 )
21


,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,학원 방역패스 제동에…청소년 접종률 상승세 꺾였다,2022.01.10. 오후 3:54,724,154,3,0,1,0,37,"[아주 잘 꺾였다, 효과도 별로 없고 부작용만 잔뜩있는 그런 물 백신, 살인 백신을..."
1,"13~18세 접종률 꺾였나…""방역패스 집행정지 영향 우려""",2022.01.10. 오후 3:51,"1,181",0,0,0,22,0,4,[3월달 애들 접종강요하면 공교육 등교거부할거다 애들 목숨이 중요하지 방학전 백신맞...
2,“비과학적” vs “백신 독려”… 해외서도 찾기 힘든 백화점 방역패스,2022.01.10. 오후 4:06,45,6,1,1,947,16,213,"[작성자에 의해 삭제된 댓글입니다., 국민들의 기본인 의식주를 원천 차단해서 불편하..."
3,"[르포] ""정말 너무한다"" 백화점·대형마트 '방역패스' 의무화 첫날…'혼란'",2022.01.10. 오후 3:43,483,31,4,3,"2,087",13,594,[투표 잘못해서 나라가 개판이 되었다 촛불이 아니라 횃불을 들고 일어나야 한다 무능...
4,대형 마트·백화점 방역패스 확대 첫 날…시민들 '불편' 목소리 커,2022.01.10. 오후 2:20,333,6,0,0,491,1,81,"[대한민국에 민주주의가 무너졌다. 공산당정부가 미쳤다!, 아니아니아니아니.아뇨 그게..."
5,"정부 ""유행 감소 방역패스 덕분…국민 불편 최소화할 것""",2022.01.10. 오전 10:09,"2,393",4,1,2,"1,024",3,454,"[거리두기때문이지 뭔 백신패스냐ㅋㅋ 매일 확진자중 미접종자 비율을 공개도 안하네, ..."
6,"권덕철 ""방역패스 확대가 자영업자 더 큰 피해 막아""",2022.01.10. 오전 9:01,411,40,5,5,"4,913",15,1750,"[이 정도면 거의 종교 아니냐 백신교, 정부 이 게 사기꾼들이 전세계 195개국 중..."
7,"권덕철, 방역패스 실효성 논란에 ""불편 최소화, 제도 조정할 것""",2022.01.10. 오전 9:10,"1,002",2,0,1,543,0,186,"[대다수가 안하는 백신패스 일부에서 하는것만 딱딱 찝어서 다 하고있는것마냥~, 방역..."
8,내일부터 백화점 · 대형마트 방역패스 실시,2022.01.09. 오전 6:16,765,54,6,2,"2,389",15,671,[문죄인정부는 정치방역으로 사람들을 완전통제하고있다 백화점이나 마트는 마스크도 벗을...
9,"촛불 든 자영업자들 ""영업제한·방역패스 철회하라""",2022.01.10. 오후 11:46,104,422,0,10,23,4,109,"[충분히 집회할만함 2년동안 참으셨으니, 딩동~ 벨소리가 한번 울릴 때마다 손님 한..."


In [26]:
res = pd.concat([r1,r2,r3]).reset_index()
res.drop('index', axis=1, inplace=True)
res

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"의대교수 등 백신인권행동, 청주 대형마트서 방역패스 항의",2022.01.10. 오후 2:24,709,923,5,1,20,14,151,[용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강...
1,청주 대형마트서 ‘방역패스’ 반대 시위… “어리석은 일”,2022.01.10. 오후 2:49,196,307,8,0,6,0,64,"[잘한다잘한다잘한다~~~, 방역패쓰 반대합니다, 응원합니다. pcr검사도 없애야 합..."
2,오늘부터 대형마트도 적용…헷갈리는 백신패스 Q&A,2022.01.10. 오전 5:32,756,5,0,2,637,2,236,"[정신병자네 마트는 안되고 지하철 버스 여객기는 되냐 ㅋㅋ, 코로나 예방을 위한 백..."
3,"""아이한테 백신 강요할까 걱정""…마트 방역패스 첫날 '불만' [현장+]",2022.01.10. 오후 9:02,"1,963",0,0,0,182,4,33,"[참 말년인데 이렇게까지 개발악하며 가는 정부는 처음 보네,퉤, 지금 백신강요는 침..."
4,"""조금 더 신중해야""…방역패스 두고 시민들 '적절' vs'부적절' 찬반",2022.01.10. 오후 1:55,296,3,0,0,229,6,71,[기사내용 중 찬성하는 한 남성... 당신이 접종했는데 뭐가 불안함 ? 그리고 접종...
...,...,...,...,...,...,...,...,...,...,...
60,내일부터 백화점·대형마트도 방역패스…16일까지 계도기간,2022.01.09. 오전 4:31,"1,506",47,3,4,"2,955",9,1085,[이재명은 방역패스 강화 한다 했고 안철수는 방역패스 철회한다 했습니다. 다들 잊지...
61,'혼자 장도 못 본다'···10일부터 마트 방역패스 시행 '부글',2022.01.09. 오전 8:02,183,126,16,16,"5,577",37,1867,[이건 심각한 인권침해에 심각한 파시즘이다.태어나서 이런 정부는 처음본다.그럼 돌파...
62,10일부터 대형마트도 방역패스 적용…1인 출입도 안돼,2022.01.09. 오전 6:38,"3,513",30,6,7,"3,655",17,1167,[사회적 거리두기는 이해가 되지만 백신패스는 과학적인 근거가 미약하다. 시행 보류해...
63,내일부터 백화점·대형마트 방역패스…17일부터 과태료 부과,2022.01.09. 오전 10:17,"1,179",31,1,2,557,3,268,[미접종자들 옹호할 생각은 없지만 이런 형평성도없고 공감도안되는 정책은 문제가 아닌...


In [34]:
cd /content/drive/MyDrive/AIBootCamp

/content/drive/MyDrive/AIBootCamp


In [96]:
res.to_csv('news_vaccine_pass.csv', encoding="utf-8-sig")

In [36]:
df = res.copy()

# 데이터 확인

In [39]:
# 중복기사 삭제
df.drop_duplicates(['기사명'])

# 댓글 수 없는 기사 삭제
df[df['댓글 수'] != 0]

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"의대교수 등 백신인권행동, 청주 대형마트서 방역패스 항의",2022.01.10. 오후 2:24,709,923,5,1,20,14,151,[용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강...
1,청주 대형마트서 ‘방역패스’ 반대 시위… “어리석은 일”,2022.01.10. 오후 2:49,196,307,8,0,6,0,64,"[잘한다잘한다잘한다~~~, 방역패쓰 반대합니다, 응원합니다. pcr검사도 없애야 합..."
2,오늘부터 대형마트도 적용…헷갈리는 백신패스 Q&A,2022.01.10. 오전 5:32,756,5,0,2,637,2,236,"[정신병자네 마트는 안되고 지하철 버스 여객기는 되냐 ㅋㅋ, 코로나 예방을 위한 백..."
3,"""아이한테 백신 강요할까 걱정""…마트 방역패스 첫날 '불만' [현장+]",2022.01.10. 오후 9:02,"1,963",0,0,0,182,4,33,"[참 말년인데 이렇게까지 개발악하며 가는 정부는 처음 보네,퉤, 지금 백신강요는 침..."
4,"""조금 더 신중해야""…방역패스 두고 시민들 '적절' vs'부적절' 찬반",2022.01.10. 오후 1:55,296,3,0,0,229,6,71,[기사내용 중 찬성하는 한 남성... 당신이 접종했는데 뭐가 불안함 ? 그리고 접종...
...,...,...,...,...,...,...,...,...,...,...
60,내일부터 백화점·대형마트도 방역패스…16일까지 계도기간,2022.01.09. 오전 4:31,"1,506",47,3,4,"2,955",9,1085,[이재명은 방역패스 강화 한다 했고 안철수는 방역패스 철회한다 했습니다. 다들 잊지...
61,'혼자 장도 못 본다'···10일부터 마트 방역패스 시행 '부글',2022.01.09. 오전 8:02,183,126,16,16,"5,577",37,1867,[이건 심각한 인권침해에 심각한 파시즘이다.태어나서 이런 정부는 처음본다.그럼 돌파...
62,10일부터 대형마트도 방역패스 적용…1인 출입도 안돼,2022.01.09. 오전 6:38,"3,513",30,6,7,"3,655",17,1167,[사회적 거리두기는 이해가 되지만 백신패스는 과학적인 근거가 미약하다. 시행 보류해...
63,내일부터 백화점·대형마트 방역패스…17일부터 과태료 부과,2022.01.09. 오전 10:17,"1,179",31,1,2,557,3,268,[미접종자들 옹호할 생각은 없지만 이런 형평성도없고 공감도안되는 정책은 문제가 아닌...


In [50]:
df = df[df['댓글 수'] != 0]
df.reset_index(drop=True,inplace=True)
df

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"의대교수 등 백신인권행동, 청주 대형마트서 방역패스 항의",2022.01.10. 오후 2:24,709,923,5,1,20,14,151,[용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강...
1,청주 대형마트서 ‘방역패스’ 반대 시위… “어리석은 일”,2022.01.10. 오후 2:49,196,307,8,0,6,0,64,"[잘한다잘한다잘한다~~~, 방역패쓰 반대합니다, 응원합니다. pcr검사도 없애야 합..."
2,오늘부터 대형마트도 적용…헷갈리는 백신패스 Q&A,2022.01.10. 오전 5:32,756,5,0,2,637,2,236,"[정신병자네 마트는 안되고 지하철 버스 여객기는 되냐 ㅋㅋ, 코로나 예방을 위한 백..."
3,"""아이한테 백신 강요할까 걱정""…마트 방역패스 첫날 '불만' [현장+]",2022.01.10. 오후 9:02,"1,963",0,0,0,182,4,33,"[참 말년인데 이렇게까지 개발악하며 가는 정부는 처음 보네,퉤, 지금 백신강요는 침..."
4,"""조금 더 신중해야""…방역패스 두고 시민들 '적절' vs'부적절' 찬반",2022.01.10. 오후 1:55,296,3,0,0,229,6,71,[기사내용 중 찬성하는 한 남성... 당신이 접종했는데 뭐가 불안함 ? 그리고 접종...
...,...,...,...,...,...,...,...,...,...,...
59,내일부터 백화점·대형마트도 방역패스…16일까지 계도기간,2022.01.09. 오전 4:31,"1,506",47,3,4,"2,955",9,1085,[이재명은 방역패스 강화 한다 했고 안철수는 방역패스 철회한다 했습니다. 다들 잊지...
60,'혼자 장도 못 본다'···10일부터 마트 방역패스 시행 '부글',2022.01.09. 오전 8:02,183,126,16,16,"5,577",37,1867,[이건 심각한 인권침해에 심각한 파시즘이다.태어나서 이런 정부는 처음본다.그럼 돌파...
61,10일부터 대형마트도 방역패스 적용…1인 출입도 안돼,2022.01.09. 오전 6:38,"3,513",30,6,7,"3,655",17,1167,[사회적 거리두기는 이해가 되지만 백신패스는 과학적인 근거가 미약하다. 시행 보류해...
62,내일부터 백화점·대형마트 방역패스…17일부터 과태료 부과,2022.01.09. 오전 10:17,"1,179",31,1,2,557,3,268,[미접종자들 옹호할 생각은 없지만 이런 형평성도없고 공감도안되는 정책은 문제가 아닌...


In [51]:
# 총 사용 데이터
df['댓글 수'].sum()

25752

In [77]:
cots = []
for comment in res['댓글'][:1]:
  for c in comment:
    print(c)

용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강제적으로 인권침해를 하고 탄압하는 빨갱이들은 모두 멸공해야 합니다! 백신패스 반대!!!
밀집도가 높은 지하철,버스는 집단감염이 나온적도없고 마스크착용을 잘하니 괜찮다.. 마트에서 장보는데누가 마스크를 벗나? 집단감염 나온적은? 과학적방역보단 정치방역을 하면서 아닌척은ㅋㅋ 근데 이제 여기는왜? 저기는왜? 할때가 아니다 백신패스 자체가 잘못되었기 때문에 모든 의구심과 부당함은 백신패스로부터였다 문제의 본질이 되는 제도를 없애지않는다면 말이 안되는 상황은 계속 일어난다 모두다 유대인이 되고싶은건가? 역사는 항상 반복되어왔다 똑같은 실수를 되풀이하지않기위해 역사를 배우지만 사람들은 멍청하게도 그사실을 금방 잊는다
마트 방역패스 적용하는 건 강제 접종 시키려는 정부의 노골적 의도이죠. 감염 확대와 큰 상관이 없고 지금까지도 한 적도 없는데 이제와서 마트가 감염가능성이 높다고 어쩌구저쩌구..황당하지..3차 접종을 강제하려는 저질스러운 수법입니다.
이게 바로 의대 교수의 품위이자 지식인으로서의 품격이다.
화이자모더나 백신 방법은 이제까지 검증이안된거라고 그렇게 얘기했는데 그리고 10년 검증이 필요한 과학적 방법이라고 몇번을 얘기했는데도 말끼를 못알아 듣네.. 이거 이대 삼대 유전자 어떻게 될지
본질적으로 마트를 못가게 하는 것 보니 백신패스는 코로나를 막기위해서가 아니라 코로나를 이용해서 통제하려는 게 골자인 것 너무 명확해지고 있습니다
이시대의 진정한 의사 이십니다 존경합니다 용기있는 의사분들 지지합니다 방역패스 제발 철회해라화이자 갑질계약에 마트도막고 백신패스 없음 일도 제데로 못하는 직군들 많아서 결국 전국민들 다맞추게 하고 부작용 오면 나몰라라 ... 미친게 아닌가 pcr검사 계속하라는건 일상생활 하지말라는거임 보건소에 병원에 사람들 우글우글 밀집되어있고 코로나 감염 위험은 더 높아지고 이게 뭔 악순환인가....??? 그냥 마스 잘쓰고 소독 잘하고 다니고 있으니까 철회해야지 맞다 의사분

## 댓글 추출

In [83]:
comments = []
for comment in res['댓글']:
  for c in comment:
    comments.append(c)

In [84]:
len(comments)

25752

In [85]:
raw_comments = pd.DataFrame(comments)
raw_comments

,0
0,용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강제...
1,"밀집도가 높은 지하철,버스는 집단감염이 나온적도없고 마스크착용을 잘하니 괜찮다.. ..."
2,마트 방역패스 적용하는 건 강제 접종 시키려는 정부의 노골적 의도이죠. 감염 확대와...
3,이게 바로 의대 교수의 품위이자 지식인으로서의 품격이다.
4,화이자모더나 백신 방법은 이제까지 검증이안된거라고 그렇게 얘기했는데 그리고 10년 ...
...,...
25747,백신 안맞는 것도 본인의 자유라매요. 백신 맞고 들어가는 것도 자유인데요? 자유에는...
25748,작성자에 의해 삭제된 댓글입니다.
25749,백신 맞던 안맞던 자유!! 하지만 미접종자 코로나 걸리면 자비로 치료!! 내 세금이...
25750,작성자에 의해 삭제된 댓글입니다.


# 삭제된 댓글 삭제

In [87]:
raw_comments[raw_comments[0] == '작성자에 의해 삭제된 댓글입니다.']

,0
97,작성자에 의해 삭제된 댓글입니다.
102,작성자에 의해 삭제된 댓글입니다.
113,작성자에 의해 삭제된 댓글입니다.
116,작성자에 의해 삭제된 댓글입니다.
121,작성자에 의해 삭제된 댓글입니다.
...,...
25741,작성자에 의해 삭제된 댓글입니다.
25742,작성자에 의해 삭제된 댓글입니다.
25745,작성자에 의해 삭제된 댓글입니다.
25748,작성자에 의해 삭제된 댓글입니다.


In [88]:
raw_comments[raw_comments[0] == '클린봇이 부적절한 표현을 감지한 댓글입니다.']

,0
98,클린봇이 부적절한 표현을 감지한 댓글입니다.
107,클린봇이 부적절한 표현을 감지한 댓글입니다.
112,클린봇이 부적절한 표현을 감지한 댓글입니다.
114,클린봇이 부적절한 표현을 감지한 댓글입니다.
115,클린봇이 부적절한 표현을 감지한 댓글입니다.
...,...
25636,클린봇이 부적절한 표현을 감지한 댓글입니다.
25643,클린봇이 부적절한 표현을 감지한 댓글입니다.
25649,클린봇이 부적절한 표현을 감지한 댓글입니다.
25683,클린봇이 부적절한 표현을 감지한 댓글입니다.


In [89]:
raw_comments = raw_comments[raw_comments[0] != '작성자에 의해 삭제된 댓글입니다.']
raw_comments.reset_index(drop=True, inplace=True)

In [90]:
raw_comments = raw_comments[raw_comments[0] != '클린봇이 부적절한 표현을 감지한 댓글입니다.']
raw_comments.reset_index(drop=True, inplace=True)

In [91]:
raw_comments

,0
0,용기내주신 모든분들 특히 의료진분들 감사합니다 자유민주주의 국가에서 이렇게까지 강제...
1,"밀집도가 높은 지하철,버스는 집단감염이 나온적도없고 마스크착용을 잘하니 괜찮다.. ..."
2,마트 방역패스 적용하는 건 강제 접종 시키려는 정부의 노골적 의도이죠. 감염 확대와...
3,이게 바로 의대 교수의 품위이자 지식인으로서의 품격이다.
4,화이자모더나 백신 방법은 이제까지 검증이안된거라고 그렇게 얘기했는데 그리고 10년 ...
...,...
21890,아직 접종안하고 버티는 사람들 대단타! 못맞는 사람이야 이해는 되지만? 불편함 감수...
21891,ㅋㅋㅋ외국에서도 다 쓰는걸 뭘 사회주의 타령이야방역패스 하니까 좋더구만 불안해 안하...
21892,백신 안맞는 것도 본인의 자유라매요. 백신 맞고 들어가는 것도 자유인데요? 자유에는...
21893,백신 맞던 안맞던 자유!! 하지만 미접종자 코로나 걸리면 자비로 치료!! 내 세금이...


In [95]:
raw_comments.to_csv('raw_comments.csv' , encoding="utf-8-sig")